In [1]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

In [3]:
import numpy as np

import nzthermo as nzt
import nzthermo.functional as F
from nzthermo.core import ParcelProfile
import metpy.calc as mpcalc
from metpy.units import units

pressure = np.array(
    [1013, 1000, 975, 950, 925, 900, 875, 850, 825, 800, 775, 750, 725, 700, 650, 600, 550, 500, 450, 400, 350, 300],
    dtype=np.float64,
)
pressure *= 100
temperature = np.array(
    [
        # [243, 242, 241, 240, 239, 237, 236, 235, 233, 232, 231, 229, 228, 226, 235, 236, 234, 231, 226, 221, 217, 211],
        # [250, 249, 248, 247, 246, 244, 243, 242, 240, 239, 238, 236, 235, 233, 240, 239, 236, 232, 227, 223, 217, 211],
        [293, 292, 290, 288, 287, 285, 284, 282, 281, 279, 279, 280, 279, 278, 275, 270, 268, 264, 260, 254, 246, 237],
        [300, 299, 297, 295, 293, 291, 292, 291, 291, 289, 288, 286, 285, 285, 281, 278, 273, 268, 264, 258, 251, 242],
    ],
    dtype=np.float64,
)
dewpoint = np.array(
    [
        # [224, 224, 224, 224, 224, 223, 223, 223, 223, 222, 222, 222, 221, 221, 233, 233, 231, 228, 223, 218, 213, 207],
        # [233, 233, 232, 232, 232, 232, 231, 231, 231, 231, 230, 230, 230, 229, 237, 236, 233, 229, 223, 219, 213, 207],
        [288, 288, 287, 286, 281, 280, 279, 277, 276, 275, 270, 258, 244, 247, 243, 254, 262, 248, 229, 232, 229, 224],
        [294, 294, 293, 292, 291, 289, 285, 282, 280, 280, 281, 281, 278, 274, 273, 269, 259, 246, 240, 241, 226, 219],
    ],
    dtype=np.float64,
)


def with_lcl(self: ParcelProfile):
    return self.pressure


def mag(x):
    return [y.m for y in x]


print(
    with_lcl(nzt.parcel_profile(pressure, temperature, dewpoint)),
    *[
        mag(
            mpcalc.parcel_profile_with_lcl(
                pressure * units.pascal,
                temperature[i] * units.degK,
                dewpoint[i] * units.degK,
            )
        )
        for i in range(temperature.shape[0])
    ],
    sep="\n"
)

[[101300.         100000.          97500.          95000.
   94064.316261    92500.          90000.          87500.
   85000.          82500.          80000.          77500.
   75000.          72500.          70000.          65000.
   60000.          55000.          50000.          45000.
   40000.          35000.          30000.        ]
 [101300.         100000.          97500.          95000.
   92815.92020834  92500.          90000.          87500.
   85000.          82500.          80000.          77500.
   75000.          72500.          70000.          65000.
   60000.          55000.          50000.          45000.
   40000.          35000.          30000.        ]]
[array([101300.        , 100000.        ,  97500.        ,  95000.        ,
        94056.05166211,  92500.        ,  90000.        ,  87500.        ,
        85000.        ,  82500.        ,  80000.        ,  77500.        ,
        75000.        ,  72500.        ,  70000.        ,  65000.        ,
        60000.  

In [6]:
from __future__ import annotations
import functools
from typing import ParamSpec, TypeVar, Callable, Concatenate, Iterable

P = ParamSpec("P")
T1 = TypeVar("T1")
T = TypeVar("T")



def map_partial(func: Callable[Concatenate[T1, P], T], __values:Iterable[T1], /, *args: P.args, **kwargs: P.kwargs) -> map[T]:
    return map(functools.partial(func, *args, **kwargs), __values)


x = list(map_partial(np.ndarray.astype, [np.array([1, 2, 3]), np.array([4, 5, 6])], dtype=np.float64))

In [ ]:

def insert(x: np.ndarray, mask: np.ndarray, value: np.ndarray):
    N, Z = x.shape
    x = np.column_stack([x, np.full(N, np.nan)])
    out = np.full_like(x, np.nan)
    # mask = zx[:, None] >= np.arange(Z)
    nx, zx = np.nonzero(mask)
    out[nx, zx - 1] = x[nx, zx]
    nx, zx = np.nonzero(~mask)
    out[nx, zx] = x[nx, zx - 1]
    out[np.isnan(out)] = value
    return out


def el(
    pressure: np.ndarray,
    temperature: np.ndarray,
    dewpoint: np.ndarray,
    parcel_profile: ParcelProfile | None = None,
    which="top",
):
    if parcel_profile is None:
        parcel_profile = nzt.parcel_profile(
            pressure, temperature[:, 0], dewpoint[:, 0], pressure[:1].repeat(temperature.shape[0])
        )
    pressure = parcel_profile.pressure
    # temperature = parcel_profile.temperature
    N, Z = temperature.shape
    lcl_index = nx, zx = parcel_profile.lcl_index
    lcl_p = parcel_profile.lcl_pressure

    # [ dewpoint ]
    mask = zx[:, None] >= np.arange(Z)
    dewpoint = insert(dewpoint, mask, parcel_profile.lcl_temperature)
    pressure = np.array([pressure] * N)

    intersect = F.intersect_nz(
        pressure,
        parcel_profile.temperature,
        np.column_stack([temperature, np.full(N, -np.inf)]),
        log_x=True,
    ).upper()
    return intersect
    # idx = x < lcl_p
    # return x, lcl_p
    return dewpoint


e = el(pressure, temperature, dewpoint)

print(e)
# print(e.x, e.y)
[
    mpcalc.el(pressure * units.pascal, temperature[i] * units.degK, dewpoint[i] * units.degK, which="bottom")
    for i in range(temperature.shape[0])
]

In [ ]:
arr = np.array([[1, 2, 3], [4, 5, 6]])

# Append a new value to each column
new_value = 7
np.column_stack((arr, [7, 7]))